In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.image import imread
import matplotlib.font_manager as fm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle

import cv2
import pickle

import time

In [ ]:
import matplotlib.pyplot as plt 
plt.rcParams.update({
    "font.family" : 'Arial',   # specify font family here
    "font.size" : 12,
    "font.weight" : "normal"})          # specify font size here

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/datas/'
file_info = pd.read_csv(base_path+'file_info.csv')

# file_info_latent
df = pd.DataFrame(( np.load(base_path+'vectors/avec_latents.npy', allow_pickle=True)),columns=['avec','Path'])
file_info_latents = pd.merge(file_info, df, how = 'left', on = 'Path')
file_info_latents = file_info_latents[~file_info_latents.avec.isnull()]

avec = np.array([i.reshape(-1) for i in file_info_latents['avec']])

In [ ]:
df = pd.DataFrame(np.load(base_path+'vectors/cvec_latents.npy', allow_pickle=True),columns=['cvec','Path'])
df['cvec']=[i.reshape(-1) for i in df['cvec']]
file_info_latents = pd.merge(file_info_latents, df, how = 'left', on = 'Path')

cvec = np.array([i.reshape(-1) for i in file_info_latents['cvec']])

## 01. UMAP 추가

In [ ]:
avec_umap = pickle.load((open(base_path+'models/avec_umap.sav', 'rb')))
cvec_umap = pickle.load((open(base_path+'models/cvec_umap.sav', 'rb')))

In [ ]:
avecs = avec_umap.transform(avec)
avecs  = pd.DataFrame(avecs,columns=['x','y'])
avecs['new_date_detail']= file_info_latents['new_date_detail']
avecs['new_date'] = list(file_info_latents['new_date'])
avecs['author_name'] = list(file_info_latents['author_name'])
avecs['Style'] = list(file_info_latents['Style'])
avecs['Path'] = list(file_info_latents['Path'])
avecs['Genre'] = list(file_info_latents['Genre'])

In [ ]:
cvecs = cvec_umap.transform(cvec)
cvecs  = pd.DataFrame(cvecs,columns=['x','y'])
cvecs['new_date_detail']= file_info_latents['new_date_detail']
cvecs['new_date']= file_info_latents['new_date']
cvecs['author_name']= file_info_latents['author_name']
cvecs['Style'] = list(file_info_latents['Style'])
cvecs['Path']= file_info_latents['Path']
cvecs['Genre']= file_info_latents['Genre']

In [ ]:
avecs['x_c'] = cvecs['x']
avecs['y_c'] = cvecs['y']

In [ ]:
#스타일 가중치 계산 및 대표 년도 추가
df_newstyle = file_info['Style'].str.split(',', expand=True)
df_newstyle = df_newstyle.melt(value_name='Style', ignore_index=False).dropna().drop(columns='variable')

temp = file_info[['new_date','Style','painting_name']].drop(columns='Style').join(df_newstyle, how='left')
temp['Weight'] = 1 / (temp.groupby('painting_name')['Style'].transform('count'))

df_newstyle = temp.dropna(subset=['Style'])

temp = df_newstyle.groupby('Style')['new_date'].agg(lambda x: x.mode().iloc[0]).reset_index()
temp.rename(columns={'new_date': 'Peak_year'}, inplace=True)

df_newstyle = df_newstyle.merge(temp, on='Style', how='left')

style_peak_year = df_newstyle[['Style','Peak_year']]
style_peak_year = style_peak_year.drop_duplicates()

new_style = list()
for i in df_newstyle['Style'] :
    year = style_peak_year['Peak_year'][style_peak_year['Style']==i].values[0]
    new_style.append(f'{i} ({year})')
df_newstyle['Style_Peak_year'] = new_style

In [ ]:
temp = df_newstyle[['Style_Peak_year','Weight']].groupby('Style_Peak_year').sum()
temp = temp.sort_values(by='Weight', ascending=False)[:10].reset_index()

styles = df_newstyle[['Style','Style_Peak_year']].drop_duplicates()
styles10 = temp['Style_Peak_year'].values

In [ ]:
df_umap_style = avecs.copy()
df_umap_style['new_style'] = df_umap_style['Style'].str.split(',')
df_umap_style = df_umap_style.explode('new_style') 

df_umap_style10 = df_umap_style[df_umap_style['new_style'].isin(styles['Style'][styles['Style_Peak_year'].isin(styles10)].unique())]
df_umap_style10['Style_Peak_year'] = df_umap_style10['new_style'].map(styles.set_index('Style')['Style_Peak_year'])

In [ ]:
#target_authors = [['Albrecht Durer (1471-1528)','albrecht durer'],['Rembrandt (1606-1669)','rembrandt'],['Johannes Vermeer (1632-1675)','johannes vermeer'],['Jean-Francois Millet (1814-1875)','jean-francois millet'],['Edouard Manet (1832-1883)','edouard manet'],['Paul Cezanne (1839-1906)','paul cezanne'],['Pierre-Auguste Renoir (1841-1919)','pierre auguste renoir'],['Vincent van Gogh (1853-1890)','vincent van gogh'],['Gustav Klimt (1862-1918)','gustav klimt'],['Egon Schiele (1890-1918)','egon schiele']]
target_authors = [['Albrecht Durer (1528)','albrecht durer'],['Rembrandt (1669)','rembrandt'],['Johannes Vermeer (1675)','johannes vermeer'],['Jean-Francois Millet (1875)','jean-francois millet'],['Edouard Manet (1883)','edouard manet'],['Paul Cezanne (1906)','paul cezanne'],['Pierre-Auguste Renoir (1919)','pierre auguste renoir'],['Vincent van Gogh (1890)','vincent van gogh'],['Gustav Klimt (1918)','gustav klimt'],['Egon Schiele (1918)','egon schiele']]

target_authors_dic = {i[1]:i[0] for i in target_authors}

df_umap_author10 = avecs[avecs['author_name'].isin(target_authors_dic.keys())]
df_umap_author10['author_name_year'] = [target_authors_dic[i] for i in df_umap_author10['author_name'] ]


### Date

In [ ]:
palette = sns.color_palette("Spectral", n_colors=7) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 4, figsize=(20, 8))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)



for idx,year in enumerate([1300,1400,1500,1600,1700,1800,1900]) :
    sns.scatterplot(x='x', y='y', data=avecs, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = avecs[avecs['new_date'].isin([year+10*i for i in range(10)])]
    sns.scatterplot(x='x', y='y', data=temp, alpha=0.6, hue_order=[1300,1400,1500,1600,1700,1800,1900], s=2, edgecolor=None, ax=ax[idx])
    
    ax[idx].set_title(f"{year} CE", fontsize=30, pad=10)
    ax[idx].set_xlim(avecs.x.min() - 0.2, avecs.x.max() + 0.2)
    ax[idx].set_ylim(avecs.y.min() - 1.2, avecs.y.max() + 1.2)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

ax[7].get_xaxis().set_visible(False)
ax[7].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_a0.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 4, figsize=(20, 8))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)



for idx,year in enumerate([1300,1400,1500,1600,1700,1800,1900]) :
    sns.scatterplot(x='x_c', y='y_c', data=avecs, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = avecs[avecs['new_date'].isin([year+10*i for i in range(10)])]
    sns.scatterplot(x='x_c', y='y_c', data=temp, alpha=0.6, hue_order=[1300,1400,1500,1600,1700,1800,1900], s=2, edgecolor=None, ax=ax[idx])
    
    ax[idx].set_title(f"{year} CE", fontsize=30, pad=10)
    ax[idx].set_xlim(avecs.x_c.min() - 0.2, avecs.x_c.max() + 0.2)
    ax[idx].set_ylim(avecs.y_c.min() - 1, avecs.y_c.max() + 0.2)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

ax[7].get_xaxis().set_visible(False)
ax[7].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_c0.png',bbox_inches='tight',transparent = True,dpi=600)

### style

In [ ]:
import matplotlib.colors as mcolors

palette = sns.color_palette("Paired") 
palette[2] = mcolors.to_rgb('#A09317')
palette[8] = mcolors.to_rgb('#872600')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

# 스타일 리스트
style_order = ['Northern Renaissance (1510)', 'Baroque (1630)', 'Rococo (1780)', 'Romanticism (1820)',
           'Realism (1880)', 'Impressionism (1880)', 'Post-Impressionism (1880)', 'Symbolism (1900)', 'Art Nouveau (Modern) (1910)',
           'Expressionism (1910)']

for idx,style in enumerate(style_order) :
    sns.scatterplot(x='x', y='y', data=df_umap_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_umap_style10[df_umap_style10['Style_Peak_year']==style]
    sns.scatterplot(x='x', y='y', data=temp, alpha=0.9, hue_order=style_order, s=2, edgecolor=None, ax=ax[idx])
    
    ax[idx].set_title(f"{style}", fontsize=20, pad=10)
    ax[idx].set_xlim(avecs.x.min() - 0.2, avecs.x.max() + 0.2)
    ax[idx].set_ylim(avecs.y.min() - 1, avecs.y.max() + 1)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_a1.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

# 스타일 리스트
styless = ['Northern Renaissance (1510)', 'Baroque (1630)', 'Rococo (1780)', 'Romanticism (1820)',
           'Realism (1880)', 'Impressionism (1880)', 'Post-Impressionism (1880)', 'Symbolism (1900)', 'Art Nouveau (Modern) (1910)',
           'Expressionism (1910)']

for idx,style in enumerate(styless) :
    sns.scatterplot(x='x_c', y='y_c', data=df_umap_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_umap_style10[df_umap_style10['Style_Peak_year']==style]
    sns.scatterplot(x='x_c', y='y_c', data=temp, alpha=0.9, hue_order=styless, s=2, edgecolor=None, ax=ax[idx])
    
    ax[idx].set_title(f"{style}", fontsize=20, pad=10)
    ax[idx].set_xlim(avecs.x_c.min() - 0.2, avecs.x_c.max() + 0.2)
    ax[idx].set_ylim(avecs.y_c.min() - 1, avecs.y_c.max() + 0.2)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_c1.png',bbox_inches='tight',transparent = True,dpi=600)

### author

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

for idx,author in enumerate(target_authors_dic.values()) :
    sns.scatterplot(x='x', y='y', data=df_umap_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_umap_author10[df_umap_author10['author_name_year']==author]
    sns.scatterplot(x='x', y='y', data=temp, s=5, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{author}", fontsize=20, pad=10)
    ax[idx].set_xlim(avecs.x.min() - 0.2, avecs.x.max() + 0.2)
    ax[idx].set_ylim(avecs.y.min() - 1, avecs.y.max() + 1)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_a2.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

for idx,author in enumerate(target_authors_dic.values()) :
    sns.scatterplot(x='x_c', y='y_c', data=df_umap_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_umap_author10[df_umap_author10['author_name_year']==author]
    sns.scatterplot(x='x_c', y='y_c', data=temp, s=5, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{author}", fontsize=20, pad=10)
    ax[idx].set_xlim(avecs.x_c.min() - 0.2, avecs.x_c.max() + 0.2)
    ax[idx].set_ylim(avecs.y_c.min() - 1, avecs.y_c.max() + 0.2)
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si05_c2.png',bbox_inches='tight',transparent = True,dpi=600)

## 02. TSNE로 추가

In [ ]:
from openTSNE import TSNE
import pickle

In [ ]:
df_tsne = pd.read_csv('datas/models/df_tsne.csv')

In [ ]:
df_tsne_style = df_tsne.copy()
df_tsne_style['new_style'] = df_tsne_style['Style'].str.split(',')
df_tsne_style = df_tsne_style.explode('new_style') 

df_tsne_style10 = df_tsne_style[df_tsne_style['new_style'].isin(styles['Style'][styles['Style_Peak_year'].isin(styles10)].unique())]
df_tsne_style10['Style_Peak_year'] = df_tsne_style10['new_style'].map(styles.set_index('Style')['Style_Peak_year'])

In [ ]:
#target_authors = [['Albrecht Durer (1471-1528)','albrecht durer'],['Rembrandt (1606-1669)','rembrandt'],['Johannes Vermeer (1632-1675)','johannes vermeer'],['Jean-Francois Millet (1814-1875)','jean-francois millet'],['Edouard Manet (1832-1883)','edouard manet'],['Paul Cezanne (1839-1906)','paul cezanne'],['Pierre-Auguste Renoir (1841-1919)','pierre auguste renoir'],['Vincent van Gogh (1853-1890)','vincent van gogh'],['Gustav Klimt (1862-1918)','gustav klimt'],['Egon Schiele (1890-1918)','egon schiele']]
target_authors = [['Albrecht Durer (1528)','albrecht durer'],['Rembrandt (1669)','rembrandt'],['Johannes Vermeer (1675)','johannes vermeer'],['Jean-Francois Millet (1875)','jean-francois millet'],['Edouard Manet (1883)','edouard manet'],['Paul Cezanne (1906)','paul cezanne'],['Pierre-Auguste Renoir (1919)','pierre auguste renoir'],['Vincent van Gogh (1890)','vincent van gogh'],['Gustav Klimt (1918)','gustav klimt'],['Egon Schiele (1918)','egon schiele']]

target_authors_dic = {i[1]:i[0] for i in target_authors}

df_tsne_author10 = df_tsne[df_tsne['author_name'].isin(target_authors_dic.keys())]
df_tsne_author10['author_name_year'] = [target_authors_dic[i] for i in df_tsne_author10['author_name'] ]


In [ ]:
import matplotlib.cm as cm

fig, ax = plt.subplots(1,1, figsize=(10, 7))
ax.patch.set_facecolor('white')

## tsne
sns.scatterplot(x='x_a',y='y_a', data=df_tsne, hue='new_date',alpha=0.5, s=3, palette="Spectral_r",linewidth=0, edgecolor=None)
ax.set_title(f"A-vector",fontsize=34, pad=20)
ax.legend(title='Date',loc = 'upper right').remove()
ax.set_xlim(df_tsne.x_a.min()-0.2, df_tsne.x_a.max()+0.2)
ax.set_ylim(df_tsne.y_a.min()-1, df_tsne.y_a.max()+1)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


## clolrbar
# ScalarMappable을 사용하여 색상 매핑을 생성합니다.
sm = cm.ScalarMappable(cmap='Spectral_r')
sm.set_array(range(70))
# colorbar를 추가합니다.
cbar = plt.colorbar(sm, ax=ax, pad=0.01, aspect=30)
cbar.set_label('Year', rotation=270, labelpad=30,fontsize=26)
cbar.outline.set_edgecolor('black')
cbar.outline.set_linewidth(0.8)
# colorbar의 눈금과 눈금 레이블을 설정합니다.
cbar.set_ticks([0,10,20,30,40,50,60, 69])  # 예시로 0, 35, 69 설정
cbar.set_ticklabels([1300,1400,1500, 1600,1700,1800, 1900,1990])  # 예시로 숫자 변경
cbar.ax.invert_yaxis()
cbar.ax.yaxis.set_tick_params(labelsize=26)


## save
fig.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_a0.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.cm as cm

fig, ax = plt.subplots(1,1, figsize=(10, 7))
ax.patch.set_facecolor('white')

## tsne
sns.scatterplot(x='x_c',y='y_c', data=df_tsne, hue='new_date',alpha=0.5, s=3, palette="Spectral_r",linewidth=0, edgecolor=None)
ax.set_title(f"C-vector",fontsize=34, pad=20)
ax.legend(title='Date',loc = 'upper right').remove()
ax.set_xlim(df_tsne.x_c.min()-0.2, df_tsne.x_c.max()+0.2)
ax.set_ylim(df_tsne.y_c.min()-1, df_tsne.y_c.max()+1)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


## clolrbar
# ScalarMappable을 사용하여 색상 매핑을 생성합니다.
sm = cm.ScalarMappable(cmap='Spectral_r')
sm.set_array(range(70))
# colorbar를 추가합니다.
cbar = plt.colorbar(sm, ax=ax, pad=0.01, aspect=30)
cbar.set_label('Year', rotation=270, labelpad=30,fontsize=26)
cbar.outline.set_edgecolor('black')
cbar.outline.set_linewidth(0.8)
# colorbar의 눈금과 눈금 레이블을 설정합니다.
cbar.set_ticks([0,10,20,30,40,50,60, 69])  # 예시로 0, 35, 69 설정
cbar.set_ticklabels([1300,1400,1500, 1600,1700,1800, 1900,1990])  # 예시로 숫자 변경
cbar.ax.invert_yaxis()
cbar.ax.yaxis.set_tick_params(labelsize=26)


## save
fig.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_c0.png',bbox_inches='tight',transparent = True,dpi=600)

### Date

In [ ]:
import matplotlib.colors as mcolors

palette = sns.color_palette("Spectral", n_colors=7) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 4, figsize=(20, 8))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)



for idx,year in enumerate([1300,1400,1500,1600,1700,1800,1900]) :
    sns.scatterplot(x='x_a', y='y_a', data=df_tsne, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne[df_tsne['new_date'].isin([year+10*i for i in range(10)])]
    sns.scatterplot(x='x_a', y='y_a', data=temp, alpha=0.6, hue_order=[1300,1400,1500,1600,1700,1800,1900], s=2, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{year} CE", fontsize=30, pad=10)
    ax[idx].set_xlim(df_tsne.x_a.min() - 0.2, df_tsne.x_a.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_a.min(), df_tsne.y_a.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

ax[7].get_xaxis().set_visible(False)
ax[7].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_a00.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 4, figsize=(20, 8))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)



for idx,year in enumerate([1300,1400,1500,1600,1700,1800,1900]) :
    sns.scatterplot(x='x_c', y='y_c', data=df_tsne, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne[df_tsne['new_date'].isin([year+10*i for i in range(10)])]
    sns.scatterplot(x='x_c', y='y_c', data=temp, alpha=0.6, hue_order=[1300,1400,1500,1600,1700,1800,1900], s=2, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{year} CE", fontsize=30, pad=10)
    ax[idx].set_xlim(df_tsne.x_c.min() - 0.2, df_tsne.x_c.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_c.min(), df_tsne.y_c.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

ax[7].get_xaxis().set_visible(False)
ax[7].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_c00.png',bbox_inches='tight',transparent = True,dpi=600)

### style

In [ ]:
import matplotlib.colors as mcolors

palette = sns.color_palette("Paired") 
palette[2] = mcolors.to_rgb('#A09317')
palette[8] = mcolors.to_rgb('#872600')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

# 스타일 리스트
styless = ['Northern Renaissance (1510)', 'Baroque (1630)', 'Rococo (1780)', 'Romanticism (1820)',
           'Realism (1880)', 'Impressionism (1880)', 'Post-Impressionism (1880)', 'Symbolism (1900)', 'Art Nouveau (Modern) (1910)',
           'Expressionism (1910)']

for idx,style in enumerate(styless) :
    sns.scatterplot(x='x_a', y='y_a', data=df_tsne_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne_style10[df_tsne_style10['Style_Peak_year']==style]
    sns.scatterplot(x='x_a', y='y_a', data=temp, alpha=0.6, hue_order=styless, s=2, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{style}", fontsize=20, pad=10)
    ax[idx].set_xlim(df_tsne.x_a.min() - 0.2, df_tsne.x_a.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_a.min(), df_tsne.y_a.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_a1.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

# 스타일 리스트
styless = ['Northern Renaissance (1510)', 'Baroque (1630)', 'Rococo (1780)', 'Romanticism (1820)',
           'Realism (1880)', 'Impressionism (1880)', 'Post-Impressionism (1880)', 'Symbolism (1900)', 'Art Nouveau (Modern) (1910)',
           'Expressionism (1910)']

for idx,style in enumerate(styless) :
    sns.scatterplot(x='x_c', y='y_c', data=df_tsne_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne_style10[df_tsne_style10['Style_Peak_year']==style]
    sns.scatterplot(x='x_c', y='y_c', data=temp, alpha=0.6, hue_order=styless, s=2, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{style}", fontsize=20, pad=10)
    ax[idx].set_xlim(df_tsne.x_c.min() - 0.2, df_tsne.x_c.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_c.min(), df_tsne.y_c.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_c1.png',bbox_inches='tight',transparent = True,dpi=600)

### author

In [ ]:
df_tsne_author10 = df_tsne[df_tsne['author_name'].isin(target_authors_dic.keys())]
df_tsne_author10['author_name_year'] = [target_authors_dic[i] for i in df_tsne_author10['author_name'] ]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

for idx,author in enumerate(target_authors_dic.values()) :
    sns.scatterplot(x='x_a', y='y_a', data=df_tsne_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne_author10[df_tsne_author10['author_name_year']==author]
    sns.scatterplot(x='x_a', y='y_a', data=temp, s=5, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{author}", fontsize=20, pad=10)
    ax[idx].set_xlim(df_tsne.x_a.min() - 0.2, df_tsne.x_a.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_a.min(), df_tsne.y_a.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_a2.png',bbox_inches='tight',transparent = True,dpi=600)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프와 축 설정 (constrained_layout 제거)
fig, ax = plt.subplots(2, 5, figsize=(21, 7))
ax = ax.reshape(-1)

# 축 설정 (배경, 테두리 등)
for i in range(2):
    ax[i].patch.set_facecolor('white')
    for spine in ax[i].spines.values():
        spine.set_color('black')
        spine.set_linewidth(0.8)

for idx,author in enumerate(target_authors_dic.values()) :
    sns.scatterplot(x='x_c', y='y_c', data=df_tsne_style, s=1, alpha=0.03, color='gray', edgecolor=None, ax=ax[idx])
    temp = df_tsne_author10[df_tsne_author10['author_name_year']==author]
    sns.scatterplot(x='x_c', y='y_c', data=temp, s=5, edgecolor=None, ax=ax[idx], color=palette[idx])
    
    ax[idx].set_title(f"{author}", fontsize=20, pad=10)
    ax[idx].set_xlim(df_tsne.x_c.min() - 0.2, df_tsne.x_c.max() + 0.2)
    ax[idx].set_ylim(df_tsne.y_c.min(), df_tsne.y_c.max())
    ax[idx].get_xaxis().set_visible(False)
    ax[idx].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si06_c2.png',bbox_inches='tight',transparent = True,dpi=600)

## 03. 벡터 거리 계산

### avec author

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import pdist, squareform
from joblib import Parallel, delayed
import random
import time

In [ ]:
## all vector
n_chunks = 100
def chunked_pdist(data_chunk):
    return pdist(data_chunk, metric='euclidean')
    
temp = np.array(list(file_info_latents['avec']))
temp = np.array_split(temp, n_chunks)

In [ ]:
avec_all = np.load(base_path+'vectors/distance/avec_all.npy') 
avec_sauthor = np.load(base_path+'vectors/distance/avec_sauthor.npy') 
avec_dauthor = np.load(base_path+'vectors/distance/avec_dauthor.npy') 

In [ ]:
avec_all_mean = np.array(avec_all).mean()
avec_sauthor_mean = np.array(avec_sauthor).mean()
avec_dauthor_mean = np.array(avec_dauthor).mean()

avec_all_var = np.array(avec_all).var()
avec_sauthor_var = np.array(avec_sauthor).var()
avec_dauthor_var = np.array(avec_dauthor).var()

print(f'avec all - var : {avec_all_var} mean : {avec_all_mean}')
print(f'avec same author - var : {avec_sauthor_var} mean : {avec_sauthor_mean}')
print(f'avec different author - var : {avec_dauthor_var} mean : {avec_dauthor_mean}')

In [ ]:
s = time.time()
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot()

## stat="density"

sns.distplot(avec_all,color='#5D5D5D', hist=False)
ax.vlines(avec_all_mean,0, 0.03, color='#000000', linestyle='solid', linewidth=1)

sns.distplot(avec_sauthor,color='#F15F5F', hist=False)
ax.vlines(avec_sauthor_mean, 0,0.03, color='#CC3D3D', linestyle='solid', linewidth=1)

sns.distplot(avec_dauthor,color='#6799FF', hist=False)
ax.vlines(avec_dauthor_mean, 0,0.03, color='#342FC7', linestyle='solid', linewidth=1)

ax.set_title("A-vector Artist Distribution",size=34 )
ax.set_xlabel("Euclidean distance",size=28)
ax.set_ylabel("Density",size=28)
ax.set_xlim(50,310)
ax.set_yticklabels(ax.get_yticklabels(),size=22)
ax.set_xticklabels(ax.get_xticklabels(),size=22)

print('time :',time.time()-s)

plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si07_a1.svg')

In [ ]:
from scipy.stats import ks_2samp

# avec_all vs avec_sauthor 비교
ks_stat, p_value = ks_2samp(avec_all, avec_sauthor)
print(f"all & same author - KS-statistic: {ks_stat}, p-value: {p_value}")

# avec_all vs avec_dauthor 비교
ks_stat, p_value = ks_2samp(avec_all, avec_dauthor)
print(f"all & diffrent author - KS-statistic: {ks_stat}, p-value: {p_value}")

# avec_sauthor vs avec_dauthor 비교
ks_stat, p_value = ks_2samp(avec_sauthor, avec_dauthor)
print(f"same author & diffrent author - KS-statistic: {ks_stat}, p-value: {p_value}")

### avec style

In [ ]:
avec_sstyle = np.load(base_path+'vectors/distance/avec_sstyle.npy') 
avec_dstyle = np.load(base_path+'vectors/distance/avec_dstyle.npy') 

In [ ]:
avec_all_mean = np.array(avec_all).mean()
avec_sstyle_mean = np.array(avec_sstyle).mean()
avec_dstyle_mean = np.array(avec_dstyle).mean()

avec_all_var = np.array(avec_all).var()
avec_sstyle_var = np.array(avec_sstyle).var()
avec_dstyle_var = np.array(avec_dstyle).var()

print(f'avec all - var : {avec_all_var} mean : {avec_all_mean}')
print(f'avec same style - var : {avec_sstyle_var} mean : {avec_sstyle_mean}')
print(f'avec different style - var : {avec_dstyle_var} mean : {avec_dstyle_mean}')

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot()

sns.distplot(avec_all,color='#5D5D5D', hist=False)
ax.vlines(avec_all_mean,0, 0.03, color='#000000', linestyle='solid', linewidth=1)

sns.distplot(avec_sstyle,color='#F15F5F', hist=False)
ax.vlines(avec_sstyle_mean, 0,0.03, color='#CC3D3D', linestyle='solid', linewidth=1)

sns.distplot(avec_dstyle,color='#6799FF', hist=False)
ax.vlines(avec_dstyle_mean, 0,0.03, color='#342FC7', linestyle='solid', linewidth=1)

ax.set_title("A-vector Style Period Distribution",size=34 )
ax.set_xlabel("Euclidean distance",size=28)
ax.set_ylabel("Density",size=28)
ax.set_xlim(50,310)
ax.set_yticklabels(ax.get_yticklabels(),size=22)
ax.set_xticklabels(ax.get_xticklabels(),size=22)

plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si07_a2.svg')

In [ ]:
from scipy.stats import ks_2samp

ks_stat, p_value = ks_2samp(avec_all, avec_sstyle)
print(f"all & same style - KS-statistic: {ks_stat}, p-value: {p_value}")

ks_stat, p_value = ks_2samp(avec_all, avec_dstyle)
print(f"all & diffrent style - KS-statistic: {ks_stat}, p-value: {p_value}")

ks_stat, p_value = ks_2samp(avec_sstyle, avec_dstyle)
print(f"same style & diffrent style - KS-statistic: {ks_stat}, p-value: {p_value}")

### cvec author

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import pdist, squareform
from joblib import Parallel, delayed
import random
import time

In [ ]:
## all vector
n_chunks = 100
def chunked_pdist(data_chunk):
    return pdist(data_chunk, metric='euclidean')
    
temp = np.array(list(file_info_latents['cvec']))
temp = np.array_split(temp, n_chunks)

In [ ]:
cvec_all = np.load(base_path+'vectors/distance/cvec_all.npy') 
cvec_sauthor = np.load(base_path+'vectors/distance/cvec_sauthor.npy') 
cvec_dauthor = np.load(base_path+'vectors/distance/cvec_dauthor.npy') 

In [ ]:
cvec_all_mean = np.array(cvec_all).mean()
cvec_sauthor_mean = np.array(cvec_sauthor).mean()
cvec_dauthor_mean = np.array(cvec_dauthor).mean()

cvec_all_var = np.array(cvec_all).var()
cvec_sauthor_var = np.array(cvec_sauthor).var()
cvec_dauthor_var = np.array(cvec_dauthor).var()

print(f'cvec all - var : {cvec_all_var} mean : {cvec_all_mean}')
print(f'cvec same author - var : {cvec_sauthor_var} mean : {cvec_sauthor_mean}')
print(f'cvec different author - var : {cvec_dauthor_var} mean : {cvec_dauthor_mean}')

In [ ]:
s = time.time()
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot()

## stat="density"

sns.distplot(cvec_all,color='#5D5D5D', hist=False)
ax.vlines(cvec_all_mean,0, 5, color='#000000', linestyle='solid', linewidth=1)

sns.distplot(cvec_sauthor,color='#F15F5F', hist=False)
ax.vlines(cvec_sauthor_mean, 0,5, color='#CC3D3D', linestyle='solid', linewidth=1)

sns.distplot(cvec_dauthor,color='#6799FF', hist=False)
ax.vlines(cvec_dauthor_mean, 0,5, color='#342FC7', linestyle='solid', linewidth=1)

ax.set_title("C-vector Artist Distribution",size=34 )
ax.set_xlabel("Euclidean distance",size=28)
ax.set_ylabel("Density",size=28)
ax.set_xlim(0,2)
ax.set_yticklabels(ax.get_yticklabels(),size=22)
ax.set_xticklabels(ax.get_xticklabels(),size=22)

print('time :',time.time()-s)

plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si07_c1.svg')

In [ ]:
from scipy.stats import ks_2samp

# cvec_all vs cvec_sauthor 비교
ks_stat, p_value = ks_2samp(cvec_all, cvec_sauthor)
print(f"all & same author - KS-statistic: {ks_stat}, p-value: {p_value}")

# cvec_all vs cvec_dauthor 비교
ks_stat, p_value = ks_2samp(cvec_all, cvec_dauthor)
print(f"all & diffrent author - KS-statistic: {ks_stat}, p-value: {p_value}")

# cvec_sauthor vs cvec_dauthor 비교
ks_stat, p_value = ks_2samp(cvec_sauthor, cvec_dauthor)
print(f"same author & diffrent author - KS-statistic: {ks_stat}, p-value: {p_value}")

### avec style

In [ ]:
cvec_sstyle = np.load(base_path+'vectors/distance/cvec_sstyle.npy') 
cvec_dstyle = np.load(base_path+'vectors/distance/cvec_dstyle.npy') 

In [ ]:
cvec_all_mean = np.array(cvec_all).mean()
cvec_sstyle_mean = np.array(cvec_sstyle).mean()
cvec_dstyle_mean = np.array(cvec_dstyle).mean()

cvec_all_var = np.array(cvec_all).var()
cvec_sstyle_var = np.array(cvec_sstyle).var()
cvec_dstyle_var = np.array(cvec_dstyle).var()

print(f'cvec all - var : {cvec_all_var} mean : {cvec_all_mean}')
print(f'cvec same style - var : {cvec_sstyle_var} mean : {cvec_sstyle_mean}')
print(f'cvec different style - var : {cvec_dstyle_var} mean : {cvec_dstyle_mean}')

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot()

sns.distplot(cvec_all,color='#5D5D5D', hist=False)
ax.vlines(cvec_all_mean,0, 6, color='#000000', linestyle='solid', linewidth=1)

sns.distplot(cvec_sstyle,color='#F15F5F', hist=False)
ax.vlines(cvec_sstyle_mean, 0,6, color='#CC3D3D', linestyle='solid', linewidth=1)

sns.distplot(cvec_dstyle,color='#6799FF', hist=False)
ax.vlines(cvec_dstyle_mean, 0,6, color='#342FC7', linestyle='solid', linewidth=1)

ax.set_title("C-vector Style Period Distribution",size=34 )
ax.set_xlabel("Euclidean distance",size=28)
ax.set_ylabel("Density",size=28)
ax.set_xlim(0,2)
ax.set_yticklabels(ax.get_yticklabels(),size=22)
ax.set_xticklabels(ax.get_xticklabels(),size=22)

plt.savefig(f'/home/jinny/projects/Art-history/Art-history/graph/supple/si07_c2.svg')

In [ ]:
from scipy.stats import ks_2samp

# cvec_all vs cvec_sauthor 비교
ks_stat, p_value = ks_2samp(cvec_all, cvec_sstyle)
print(f"all & same style - KS-statistic: {ks_stat}, p-value: {p_value}")

# cvec_all vs cvec_dauthor 비교
ks_stat, p_value = ks_2samp(cvec_all, cvec_dstyle)
print(f"all & diffrent style - KS-statistic: {ks_stat}, p-value: {p_value}")

# cvec_sauthor vs cvec_dauthor 비교
ks_stat, p_value = ks_2samp(cvec_sstyle, cvec_dstyle)
print(f"same style & diffrent style - KS-statistic: {ks_stat}, p-value: {p_value}")